# Imports

In [1]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import networkx as nx
from node2vec import Node2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import average_precision_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score

C:\Users\Ayan\anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Ayan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.GK7GX5KEQ4F6UYO3P26ULGBQYHGQO7J4.gfortran-win_amd64.dll
C:\Users\Ayan\anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


# Load data

In [2]:
soc_reddit_body = pd.read_csv('soc-redditHyperlinks-body.tsv', sep='\t')

In [3]:
soc_reddit_body.head()

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08..."
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049..."
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082..."
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0...."
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0...."


In [4]:
embed_subreddit = pd.read_csv('web-redditEmbeddings-subreddits.csv')

In [5]:
embed_subreddit.head()

,spiders,0.158972,0.285813,0.226329,-0.183338,-0.163159,-0.18891,0.422705,0.168602,0.178193,...,-0.025041,0.184655,0.111942,-1.125799,0.223129,-0.387323,-0.234882,-0.252297,-0.054315,0.500009
0,askreddit,-0.499114,0.323983,-0.424809,-0.222705,-0.327201,0.412856,1.040088,0.996697,-1.256240,...,0.072151,0.226658,0.535705,0.039861,-0.027087,-0.239952,0.554510,0.676439,-0.328846,0.941547
1,globaloffensivetrade,-0.023145,-1.199374,1.661484,-1.025296,1.424670,-1.179507,-1.035703,0.329301,-0.567272,...,2.998430,-1.235703,-0.326850,4.381238,-1.965378,-2.369674,-0.615389,2.010601,-0.919314,-2.149625
2,fireteams,2.492506,-2.529917,-0.448484,-3.543441,-0.586122,-0.101517,1.837567,-0.215715,-2.460388,...,-1.875214,0.969291,0.497958,2.941132,2.423561,1.315691,-0.959618,0.151845,-0.852492,-0.467526
3,funny,-0.819370,-0.865261,0.301753,0.018787,0.201029,0.070462,0.903895,1.038460,-0.306950,...,1.364194,0.830233,-0.137822,-2.204667,-0.162008,0.390291,-0.556404,0.322684,0.641634,1.320716
4,the_donald,-0.123265,-0.610208,0.361495,-1.171773,0.367407,0.624919,2.420601,-0.447998,1.393862,...,0.981761,1.044754,-2.334628,-0.005856,-1.647531,-0.139574,-0.284726,0.315069,-0.846018,0.800535


In [6]:
nodes = set(soc_reddit_body['SOURCE_SUBREDDIT'].tolist()).union(set(soc_reddit_body['TARGET_SUBREDDIT'].tolist()))

In [7]:
len(nodes)

35776

In [8]:
node_emb_names = []
node_embd_vec = []

for j in tqdm(range(len(embed_subreddit))):
    node_emb_names.append(embed_subreddit.iloc[j].tolist()[0])
    node_embd_vec.append(embed_subreddit.iloc[j].tolist()[1:302])

100%|██████████████████████████████████████████████████████████████████████████| 51277/51277 [00:23<00:00, 2222.74it/s]


In [9]:
node_embd_name_vec = dict(zip(node_emb_names,node_embd_vec))

In [10]:
len(node_emb_names)

51277

In [11]:
len(nodes.intersection(set(node_emb_names)))

21674

In [12]:
node_1_embed = []
node_2_embed = []
year_list = []
found_1 = []
found_2 = []

for index, row in tqdm(soc_reddit_body.iterrows()):
    year_list.append(row['TIMESTAMP'].split()[0].split('-')[0])
    try:
        node_1_embed.append(node_embd_name_vec[row['SOURCE_SUBREDDIT']])
        found_1.append(1)
    except:
        node_1_embed.append([0]*300)
        found_1.append(0)
    try:
        node_2_embed.append(node_embd_name_vec[row['TARGET_SUBREDDIT']])
        found_2.append(1)
    except:
        node_2_embed.append([0]*300)
        found_2.append(0)

print(set(year_list))

286561it [00:31, 9161.95it/s]

{'2016', '2013', '2015', '2014', '2017'}


In [13]:
soc_reddit_body['year'] = year_list
soc_reddit_body['source_embed'] = node_1_embed
soc_reddit_body['target_embed'] = node_2_embed
soc_reddit_body['found_1'] = found_1
soc_reddit_body['found_2'] = found_2

In [14]:
soc_reddit_body

,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,POST_ID,TIMESTAMP,LINK_SENTIMENT,PROPERTIES,year,source_embed,target_embed,found_1,found_2
0,leagueoflegends,teamredditteams,1u4nrps,2013-12-31 16:39:58,1,"345.0,298.0,0.75652173913,0.0173913043478,0.08...",2013,"[-2.785298, -0.166391, 1.592624, -1.269829, 2....","[-1.402933, -1.1156540000000001, 0.921651, -0....",1,1
1,theredlion,soccer,1u4qkd,2013-12-31 18:18:37,-1,"101.0,98.0,0.742574257426,0.019801980198,0.049...",2013,"[-0.239977, -0.24648299999999998, 0.0596789999...","[-2.7988470000000003, -1.243099, 0.513319, 0.1...",1,1
2,inlandempire,bikela,1u4qlzs,2014-01-01 14:54:35,1,"85.0,85.0,0.752941176471,0.0235294117647,0.082...",2014,"[0.070123, -0.001389, 0.143414, -0.46698900000...","[0.15803, 0.035602999999999996, 0.197221, -0.4...",1,1
3,nfl,cfb,1u4sjvs,2013-12-31 17:37:55,1,"1124.0,949.0,0.772241992883,0.0017793594306,0....",2013,"[-0.41374399999999995, 0.13559300000000002, -0...","[-0.5422899999999999, 1.0747719999999998, 0.23...",1,1
4,playmygame,gamedev,1u4w5ss,2014-01-01 02:51:13,1,"715.0,622.0,0.777622377622,0.00699300699301,0....",2014,"[0.006368, -0.135252, 0.45853299999999997, -0....","[-0.033949, 0.15096600000000002, 1.039643, -0....",1,1
...,...,...,...,...,...,...,...,...,...,...,...
286556,negareddit,debatefascism,68im20s,2017-04-30 16:31:26,1,"441.0,405.0,0.775510204082,0.0294784580499,0.0...",2017,"[-0.392242, 0.186111, 0.203193, -0.441168, 0.3...","[-0.202682, 0.126838, -0.077966, -0.2579079999...",1,1
286557,mildlynomil,justnomil,68imlas,2017-04-30 04:19:03,1,"2226.0,1855.0,0.786163522013,0.00224618149146,...",2017,"[-0.008284, -0.075661, 0.082401, -0.049988, 0....","[0.124855, -0.792166, 0.149331, 0.376434999999...",1,1
286558,mmorpg,blackdesertonline,68ip5os,2017-04-30 16:54:08,1,"1100.0,909.0,0.778181818182,0.00181818181818,0...",2017,"[0.008334999999999999, 0.0209, 0.9342010000000...","[-0.272359, -0.833299, 1.339546, -0.268322, 0....",1,1
286559,electricskateboards,askreddit,68ipb2s,2017-04-30 16:41:53,1,"1876.0,1567.0,0.78144989339,0.00692963752665,0...",2017,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[-0.49911400000000006, 0.32398299999999997, -0...",0,1


# Trains on 2016 and test on 2017

In [15]:
train = soc_reddit_body[soc_reddit_body['year'] == '2016']
test = soc_reddit_body[soc_reddit_body['year'] == '2017']
print(len(train),len(test))

98229 36406


In [16]:
train_nodes = set(train['SOURCE_SUBREDDIT'].tolist()).union(set(train['TARGET_SUBREDDIT'].tolist()))
test_nodes = set(test['SOURCE_SUBREDDIT'].tolist()).union(set(test['TARGET_SUBREDDIT'].tolist()))
print('Inductive nodes: ', len(test_nodes.difference(train_nodes)))

Inductive nodes:  3758


In [17]:
inductive_nodes = test_nodes.difference(train_nodes)
inductive_test = test[test['SOURCE_SUBREDDIT'].isin(inductive_nodes) | test['TARGET_SUBREDDIT'].isin(inductive_nodes)]
print('Size of inductive test set: ', len(inductive_test))

Size of inductive test set:  7035


In [18]:
train_valid = train[train['found_1'].isin([1]) & train['found_2'].isin([1])]
print(len(train_valid))
inductive_test_valid = inductive_test[inductive_test['found_1'].isin([1]) & inductive_test['found_2'].isin([1])]
print(len(inductive_test_valid))

87401
3795


In [19]:
train_valid = train_valid[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT', 'year', 'source_embed', 'target_embed']]
inductive_test_valid = inductive_test_valid[['SOURCE_SUBREDDIT','TARGET_SUBREDDIT', 'year', 'source_embed', 'target_embed']]

## Generating negatives

In [20]:
train_valid_node_dict = dict()

for index, row in tqdm(train_valid.iterrows()):
    train_valid_node_dict[row['SOURCE_SUBREDDIT']] = row['source_embed']
    train_valid_node_dict[row['TARGET_SUBREDDIT']] = row['target_embed']
    
inductive_test_valid_node_dict = dict()

for index, row in tqdm(inductive_test_valid.iterrows()):
    inductive_test_valid_node_dict[row['SOURCE_SUBREDDIT']] = row['source_embed']
    inductive_test_valid_node_dict[row['TARGET_SUBREDDIT']] = row['target_embed']

87401it [00:10, 8037.09it/s]
3795it [00:00, 7846.85it/s]


In [21]:
train_neg = pd.DataFrame()

source_list = []
target_list = []
year_list = []
source_embed_list = []
target_embed_list = []

for j in tqdm(range(len(train_valid))):
    
    src = np.random.choice(list(train_valid_node_dict.keys()))
    trgt = np.random.choice(list(train_valid_node_dict.keys()))
    
    source_list.append(src)
    target_list.append(trgt)
    year_list.append(2016)
    source_embed_list.append(train_valid_node_dict[src])
    target_embed_list.append(train_valid_node_dict[trgt])
    
train_neg['SOURCE_SUBREDDIT'] = source_list
train_neg['TARGET_SUBREDDIT'] = target_list
train_neg['year'] = year_list
train_neg['source_embed'] = source_embed_list
train_neg['target_embed'] = target_embed_list

train_all = pd.concat([train_valid,train_neg])
train_all['label'] = [1] * len(train_valid) + [0] * len(train_neg)
train_all

100%|███████████████████████████████████████████████████████████████████████████| 87401/87401 [09:59<00:00, 145.72it/s]


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,year,source_embed,target_embed,label
128697,stevenuniversefanfic,stevenuniverse,2016,"[-0.028525, -0.053530999999999995, 0.085959000...","[-0.573696, -0.48343, 1.3551090000000001, 0.37...",1
135255,tworedditorsonecup,formula1,2016,"[-0.015385, -0.070425, 0.076922, -0.060391, 0....","[0.385633, -0.9249379999999999, 0.239616, 0.72...",1
148286,listofsubreddits,changelog,2016,"[0.000697, -0.065453, 0.060517999999999995, -0...","[-0.02054, -0.07195599999999999, 0.074525, -0....",1
148411,faggots,redditrequest,2016,"[0.001968, 0.0008300000000000001, -0.001843, -...","[-0.001361, 0.36938899999999997, 0.024178, -0....",1
149389,tppthegame,twitchplayspokemon,2016,"[-0.095835, -0.069751, 0.116627, -0.004266, 0....","[-1.164833, -0.03354, 0.916997, -0.47464799999...",1
...,...,...,...,...,...,...
87396,shittysteamreviews,tytsucks,2016,"[-0.041770999999999996, -0.101991, 0.107123, -...","[-0.099248, -0.062863, 0.048165, -0.105651, 0....",0
87397,mushishi,alcoholism,2016,"[-0.025792000000000002, -0.055833, 0.089972, -...","[0.052161, 0.0511, 0.181901, -0.503351, -0.025...",0
87398,stem,gopro,2016,"[-0.016462, -0.07764299999999999, 0.075554, -0...","[0.41653, -1.026502, 0.8841129999999999, -0.34...",0
87399,designerreps,themarketsofsidon,2016,"[-0.206596, -0.718169, 0.26658899999999996, 0....","[-0.006553, -0.08631799999999999, 0.085161, -0...",0


In [22]:
inductive_test_valid_neg = pd.DataFrame()

source_list = []
target_list = []
year_list = []
source_embed_list = []
target_embed_list = []

for j in tqdm(range(len(inductive_test_valid))):
    
    src = np.random.choice(list(inductive_test_valid_node_dict.keys()))
    trgt = np.random.choice(list(inductive_test_valid_node_dict.keys()))
    
    source_list.append(src)
    target_list.append(trgt)
    year_list.append(2017)
    source_embed_list.append(inductive_test_valid_node_dict[src])
    target_embed_list.append(inductive_test_valid_node_dict[trgt])
    
inductive_test_valid_neg['SOURCE_SUBREDDIT'] = source_list
inductive_test_valid_neg['TARGET_SUBREDDIT'] = target_list
inductive_test_valid_neg['year'] = year_list
inductive_test_valid_neg['source_embed'] = source_embed_list
inductive_test_valid_neg['target_embed'] = target_embed_list

test_all = pd.concat([inductive_test_valid,inductive_test_valid_neg])
test_all['label'] = [1] * len(inductive_test_valid) + [0] * len(inductive_test_valid_neg)
test_all

100%|█████████████████████████████████████████████████████████████████████████████| 3795/3795 [00:06<00:00, 591.22it/s]


,SOURCE_SUBREDDIT,TARGET_SUBREDDIT,year,source_embed,target_embed,label
232949,islamwatch,islam,2017,"[-0.025353, -0.048428, 0.036367000000000003, -...","[-0.134543, 0.828586, -0.418367, -0.511281, -0...",1
241752,westshire,lordsofminecraft,2017,"[-0.007277, -0.089598, 0.079178, -0.06075, 0.0...","[0.030913, -0.270242, 0.221529, -0.101273, 0.1...",1
247402,socialistamerica,vexillology,2017,"[-0.046465, -0.058859, 0.07697899999999999, -0...","[-0.591905, -0.361043, 0.107599, -0.842623, 0....",1
249591,capitalismiscrumbling,latestagecapitalism,2017,"[-0.016158000000000002, -0.050200999999999996,...","[-0.364487, 0.321563, -0.015908000000000002, -...",1
250062,rgbprofiles,programming,2017,"[-0.01441, -0.064221, 0.085774, -0.056686, 0.0...","[-0.071564, 0.007873999999999999, 0.9212520000...",1
...,...,...,...,...,...,...
3790,olympicgamesbids,entrepreneur,2017,"[-0.046683999999999996, -0.028916000000000004,...","[-0.742021, -1.3089309999999998, 0.464033, -1....",0
3791,conspiracytheories,ashemains,2017,"[0.005628, -0.036646, 0.050834, -0.072123, 0.0...","[-0.031523, -0.08709800000000001, 0.089517, -0...",0
3792,gilmoregirls,researchmarkets,2017,"[0.105679, -0.329646, 0.160098, -0.098077, -0....","[-0.04262, -0.215193, 0.027936000000000002, -0...",0
3793,sonics,wifesharing,2017,"[-0.031049, -0.050687, 0.057935, -0.1363650000...","[0.5246970000000001, -0.795682, -0.209837, 0.2...",0


# Train and test classifier

In [23]:
X_train = []

for index, row in tqdm(train_all.iterrows()):
    X_train.append(row['source_embed'] + row['target_embed'])
    
X_test = []

for index, row in tqdm(test_all.iterrows()):
    X_test.append(row['source_embed'] + row['target_embed'])

174802it [00:25, 6955.70it/s]
7590it [00:01, 7537.63it/s]


In [24]:
clf = MLPClassifier(hidden_layer_sizes=(3,100),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

In [25]:
y_train = train_all['label'].tolist()
clf.fit(X_train,y_train)

Iteration 1, loss = 0.38646726
Iteration 2, loss = 0.35508187
Iteration 3, loss = 0.34898923
Iteration 4, loss = 0.34502568
Iteration 5, loss = 0.34234829
Iteration 6, loss = 0.34108906
Iteration 7, loss = 0.34028463
Iteration 8, loss = 0.33808593
Iteration 9, loss = 0.33794647
Iteration 10, loss = 0.33666981
Iteration 11, loss = 0.33632603
Iteration 12, loss = 0.33451489
Iteration 13, loss = 0.33465073
Iteration 14, loss = 0.33371436
Iteration 15, loss = 0.33323708
Iteration 16, loss = 0.33297064
Iteration 17, loss = 0.33262955
Iteration 18, loss = 0.33235074
Iteration 19, loss = 0.33236207
Iteration 20, loss = 0.33118183
Iteration 21, loss = 0.33109988
Iteration 22, loss = 0.33066966
Iteration 23, loss = 0.33215095
Iteration 24, loss = 0.33229352
Iteration 25, loss = 0.32879324
Iteration 26, loss = 0.32907172
Iteration 27, loss = 0.32978570
Iteration 28, loss = 0.32867858
Iteration 29, loss = 0.32853864
Iteration 30, loss = 0.32802449
Iteration 31, loss = 0.32822399
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(3, 100), learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=5, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [26]:
ypred = clf.predict(X_test)

In [27]:
print('AUROC:', roc_auc_score(test_all['label'].tolist(), ypred))
print('AUPRC:', average_precision_score(test_all['label'].tolist(), ypred))

AUROC: 0.6391304347826088
AUPRC: 0.5970992764487322


# Random Shuffling

In [28]:
def crazyshuffle(arr):
    
    x, y = arr.shape
    rows = np.indices((x,y))[0]
    cols = [np.random.permutation(y) for _ in range(x)]
    
    return arr[rows, cols]

In [29]:
X_train_shuffle = np.array(X_train)
X_train_shuffle = crazyshuffle(X_train_shuffle)

clf_shuffled = MLPClassifier(hidden_layer_sizes=(3,100),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

clf_shuffled.fit(X_train_shuffle,y_train)


Iteration 1, loss = 0.56349230
Iteration 2, loss = 0.53806371
Iteration 3, loss = 0.52963832
Iteration 4, loss = 0.52416644
Iteration 5, loss = 0.52147878
Iteration 6, loss = 0.51985873
Iteration 7, loss = 0.51633925
Iteration 8, loss = 0.51437721
Iteration 9, loss = 0.51430080
Iteration 10, loss = 0.51231388
Iteration 11, loss = 0.51213990
Iteration 12, loss = 0.51031313
Iteration 13, loss = 0.50985105
Iteration 14, loss = 0.50917845
Iteration 15, loss = 0.50985630
Iteration 16, loss = 0.50809664
Iteration 17, loss = 0.50732600
Iteration 18, loss = 0.50814759
Iteration 19, loss = 0.50722837
Iteration 20, loss = 0.50621796
Iteration 21, loss = 0.50635019
Iteration 22, loss = 0.50597834
Iteration 23, loss = 0.50569931
Iteration 24, loss = 0.50528603
Iteration 25, loss = 0.50490272
Iteration 26, loss = 0.50480047
Iteration 27, loss = 0.50429325
Iteration 28, loss = 0.50438614
Iteration 29, loss = 0.50420685
Iteration 30, loss = 0.50429298
Iteration 31, loss = 0.50388534
Iteration 32, los

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=False, epsilon=1e-08,
              hidden_layer_sizes=(3, 100), learning_rate='constant',
              learning_rate_init=0.01, max_fun=15000, max_iter=200,
              momentum=0.9, n_iter_no_change=10, nesterovs_momentum=True,
              power_t=0.5, random_state=5, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=True,
              warm_start=False)

In [30]:
ypred_shuffle = clf_shuffled.predict(X_test)

print('AUROC:', roc_auc_score(test_all['label'].tolist(), ypred_shuffle))
print('AUPRC:', average_precision_score(test_all['label'].tolist(), ypred_shuffle))

AUROC: 0.5956521739130435
AUPRC: 0.5566836734693877


# Random features

In [31]:
X_train_rand = np.random.rand(np.array(X_train).shape[0],np.array(X_train).shape[1])

clf_rand = MLPClassifier(hidden_layer_sizes=(3,100),
                    random_state=5,
                    verbose=True,
                    learning_rate_init=0.01)

clf_rand.fit(X_train_rand,y_train)

ypred_rand = clf_rand.predict(X_test)

print('AUROC:', roc_auc_score(test_all['label'].tolist(), ypred_rand))
print('AUPRC:', average_precision_score(test_all['label'].tolist(), ypred_rand))

Iteration 1, loss = 0.69354510
Iteration 2, loss = 0.69322458
Iteration 3, loss = 0.69327081
Iteration 4, loss = 0.69323522
Iteration 5, loss = 0.69322411
Iteration 6, loss = 0.69321293
Iteration 7, loss = 0.69323157
Iteration 8, loss = 0.69330053
Iteration 9, loss = 0.69326038
Iteration 10, loss = 0.69326591
Iteration 11, loss = 0.69324002
Iteration 12, loss = 0.69318859
Iteration 13, loss = 0.69321564
Training loss did not improve more than tol=0.000100 for 10 consecutive epochs. Stopping.
AUROC: 0.5
AUPRC: 0.5


# Davis-Bouldin Score

In [32]:
k_clusters = 5

attributes_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train)
print(davies_bouldin_score(X_train,attributes_kmeans.labels_))
      
shuffle_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_shuffle)
print(davies_bouldin_score(X_train_shuffle,shuffle_kmeans.labels_))
      
rand_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_rand)
print(davies_bouldin_score(X_train_rand,rand_kmeans.labels_))

2.612767439715917
15.043791458449059
22.30188751914002


In [33]:
k_clusters = 10

attributes_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train)
print(davies_bouldin_score(X_train,attributes_kmeans.labels_))
      
shuffle_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_shuffle)
print(davies_bouldin_score(X_train_shuffle,shuffle_kmeans.labels_))
      
rand_kmeans = KMeans(n_clusters=k_clusters, random_state=0).fit(X_train_rand)
print(davies_bouldin_score(X_train_rand,rand_kmeans.labels_))

2.3093943753283983
10.640470887485638
18.85099979497361
